In [ ]:
pip install nltk --quiet

In [ ]:
import pandas as pd                               #importamos las librerias necesarias
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize

In [ ]:
df = pd.read_csv('fgj.csv') #Cargamos el archivo original

<ipython-input-7-85d9a39430db>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('fgj.csv')


In [ ]:
df.head()

,anio_inicio,mes_inicio,fecha_inicio,hora_inicio,anio_hecho,mes_hecho,fecha_hecho,hora_hecho,delito,categoria_delito,...,fiscalia,agencia,unidad_investigacion,colonia_hecho,colonia_catalogo,alcaldia_hecho,alcaldia_catalogo,municipio_hecho,latitud,longitud
0,2016,Enero,2016-01-01,00:00:00,2015.0,Diciembre,2015-12-31,16:30:00,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,DELITO DE BAJO IMPACTO,...,INVESTIGACIÓN EN TLALPAN,TLP-4,UI-2CD,JARDINES EN LA MONTAÑA,Jardines En La Montaña,TLALPAN,Tlalpan,CDMX,19.30086,-99.20877
1,2016,Enero,2016-01-01,00:00:00,2015.0,Diciembre,2015-12-31,22:40:00,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,...,INVESTIGACIÓN EN TLALPAN,TLP-1,UI-2CD,LOMAS DE PADIERNA,Lomas De Padierna,TLALPAN,Tlalpan,CDMX,19.29003,-99.21748
2,2016,Enero,2016-01-01,00:00:00,2016.0,Enero,2016-01-01,00:20:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,...,INVESTIGACIÓN EN IZTAPALAPA,IZP-2,UI-2CD,SAN ANTONIO CULHUACÁN,Barrio San Antonio Culhuacan,IZTAPALAPA,Iztapalapa,CDMX,19.34080,-99.11431
3,2016,Enero,2016-01-01,00:00:00,2015.0,Diciembre,2015-12-31,22:00:00,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,...,INVESTIGACIÓN EN GUSTAVO A. MADERO,GAM-8,UI-2CD,SAN JUAN DE ARAGÓN II SECCIÓN,San Juan De Aragon Ii Seccion,GUSTAVO A. MADERO,Gustavo A. Madero,CDMX,19.45106,-99.08669
4,2016,Enero,2016-01-01,00:00:00,2015.0,Diciembre,2015-12-31,22:30:00,HOMICIDIOS INTENCIONALES (OTROS),HOMICIDIO DOLOSO,...,INVESTIGACIÓN EN BENITO JUÁREZ,BJ-1,UI-2SD,NATIVITAS,Nativitas,BENITO JUAREZ,Benito Juárez,CDMX,19.38095,-99.13982


In [ ]:
df.isna().sum() # vemos si hay nulos

,0
anio_inicio,0
mes_inicio,0
fecha_inicio,3
hora_inicio,15
anio_hecho,545
mes_hecho,545
fecha_hecho,546
hora_hecho,885
delito,0
categoria_delito,0


In [ ]:
for col in df.columns: # convertimos las fechas a tipo datetime
    if col == 'fecha_inicio' or col == 'fecha_hecho':
        df[col] = pd.to_datetime(df[col], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2025785 entries, 0 to 2025784
Data columns (total 21 columns):
 #   Column                Dtype         
---  ------                -----         
 0   anio_inicio           int64         
 1   mes_inicio            object        
 2   fecha_inicio          datetime64[ns]
 3   hora_inicio           object        
 4   anio_hecho            float64       
 5   mes_hecho             object        
 6   fecha_hecho           datetime64[ns]
 7   hora_hecho            object        
 8   delito                object        
 9   categoria_delito      object        
 10  competencia           object        
 11  fiscalia              object        
 12  agencia               object        
 13  unidad_investigacion  object        
 14  colonia_hecho         object        
 15  colonia_catalogo      object        
 16  alcaldia_hecho        object        
 17  alcaldia_catalogo     object        
 18  municipio_hecho       object        
 19  

In [ ]:
df['mes_hecho'] = df['fecha_hecho'].dt.month #Separamos los meses años
df['anio_hecho'] = df['fecha_hecho'].dt.year

df['mes_inicio'] = df['fecha_inicio'].dt.month
df['anio_inicio'] = df['fecha_inicio'].dt.year



df['hora_hecho'] = df['hora_hecho'].apply(lambda x: int(x.split(':')[0]) if not pd.isna(x) else x)    #Convertimos a datos numéricos las hora
df['hora_inicio'] = df['hora_inicio'].apply(lambda x: int(x.split(':')[0]) if not pd.isna(x) else x)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2025785 entries, 0 to 2025784
Data columns (total 21 columns):
 #   Column                Dtype         
---  ------                -----         
 0   anio_inicio           float64       
 1   mes_inicio            float64       
 2   fecha_inicio          datetime64[ns]
 3   hora_inicio           float64       
 4   anio_hecho            float64       
 5   mes_hecho             float64       
 6   fecha_hecho           datetime64[ns]
 7   hora_hecho            float64       
 8   delito                object        
 9   categoria_delito      object        
 10  competencia           object        
 11  fiscalia              object        
 12  agencia               object        
 13  unidad_investigacion  object        
 14  colonia_hecho         object        
 15  colonia_catalogo      object        
 16  alcaldia_hecho        object        
 17  alcaldia_catalogo     object        
 18  municipio_hecho       object        
 19  

In [ ]:
df_numerics = df.select_dtypes(include=['int64', 'float64'])    # HAcemos un df con solo datos numéricos

In [ ]:
df_numerics.isna().sum() #Visualizamos los datos nulos de los numéricos

,0
anio_inicio,3
mes_inicio,3
hora_inicio,15
anio_hecho,546
mes_hecho,546
hora_hecho,885
latitud,96714
longitud,96714


In [ ]:
for col in df_numerics.columns:     # Llenamos los datos de fecha
    if (col != 'latitud') & (col != 'longitud'):
        df_numerics[col] = df_numerics[col].ffill()

    df[col] = df_numerics[col]

df.isna().sum()

,0
anio_inicio,0
mes_inicio,0
fecha_inicio,3
hora_inicio,0
anio_hecho,0
mes_hecho,0
fecha_hecho,546
hora_hecho,0
delito,0
categoria_delito,0


In [ ]:
for col in df.columns:          #Llenamos los datos de fecha
    if col == 'fecha_inicio' or col == 'fecha_hecho':
        df[col] = df[col].ffill()
df.isna().sum()

,0
anio_inicio,0
mes_inicio,0
fecha_inicio,0
hora_inicio,0
anio_hecho,0
mes_hecho,0
fecha_hecho,0
hora_hecho,0
delito,0
categoria_delito,0


In [ ]:
df['colonia_catalogo'].value_counts()     # Mostramos los valores y el conteo de columnas categóricas

,count
colonia_catalogo,
Centro,62939
Doctores,37353
Roma Norte,23056
Buenavista,17920
Morelos,17543
...,...
Tepantitlamilco,14
Canutillo 3ra Seccion,12
Chichilaula,11


In [ ]:
df['alcaldia_catalogo'].value_counts()

,count
alcaldia_catalogo,
Cuauhtémoc,288872
Iztapalapa,283172
Gustavo A. Madero,196278
Benito Juárez,156750
Coyoacán,134973
Álvaro Obregón,131812
Miguel Hidalgo,124112
Tlalpan,119008
Venustiano Carranza,111869


In [ ]:
df['municipio_hecho'].value_counts()

,count
municipio_hecho,
CDMX,2002384
NEZAHUALCOYOTL,2395
TLALNEPANTLA DE BAZ,2231
ECATEPEC DE MORELOS,1944
NAUCALPAN DE JUAREZ,1634
...,...
SANTA ANA,1
IRIMBO,1
AMATENANGO DEL VALLE,1


In [ ]:
df['fiscalia'].value_counts()

,count
fiscalia,
AGENCIA DE DENUNCIA DIGITAL,142081
INVESTIGACIÓN EN IZTAPALAPA,114936
INVESTIGACIÓN EN CUAUHTEMOC,114402
FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN IZTAPALAPA,109417
FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN CUAUHTÉMOC,92414
...,...
FISCALÍA ESPECIALIZADA PARA LA ATENCIÓN DE DELITOS ELECTORALES\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\n,3
FISCALÍA ESPECIALIZADA PARA LA ATENCIÓN DE DELITOS ELECTORALES\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\n,3
FISCALÍA ESPECIALIZADA PARA LA ATENCIÓN DE DELITOS ELECTORALES\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\n,3


In [ ]:
df['agencia'].value_counts()

,count
agencia,
CEN-1,72170
CUH-2,54814
IZP-6,52631
B,51059
IZP-8,41224
...,...
AZ-3,1
URI-GAM-4,1
HOM-E,1


In [ ]:
df['competencia'].value_counts()

,count
competencia,
FUERO COMUN,931132
HECHO NO DELICTIVO,19428
INCOMPETENCIA,11207


In [ ]:
df['unidad_investigacion'].value_counts()

,count
unidad_investigacion,
UI-1SD,698621
UI-1CD,218970
UI-2SD,211800
UI-3CD,203044
UI-2CD,198267
...,...
Unidad de investigación AM-3 S/D,1
Unidad de investigación IND-S/D,1
FDS-4-04,1


In [ ]:
df['delito'].value_counts()

,count
delito,
VIOLENCIA FAMILIAR,250385
FRAUDE,149643
AMENAZAS,129841
ROBO DE OBJETOS,109858
ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,86339
...,...
INJURIAS,1
CONTRA LA LEY FEDERAL DE POBLACION,1
HOMICIDIO POR INMERSION,1


In [ ]:
df_c = df.copy() #Generamos una copia para para manejar los datos correctamente

In [ ]:
df_c.head()

,anio_inicio,mes_inicio,fecha_inicio,hora_inicio,anio_hecho,mes_hecho,fecha_hecho,hora_hecho,delito,categoria_delito,...,fiscalia,agencia,unidad_investigacion,colonia_hecho,colonia_catalogo,alcaldia_hecho,alcaldia_catalogo,municipio_hecho,latitud,longitud
0,2016.0,1.0,2016-01-01,0.0,2015.0,12.0,2015-12-31,16.0,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,DELITO DE BAJO IMPACTO,...,INVESTIGACIÓN EN TLALPAN,TLP-4,UI-2CD,JARDINES EN LA MONTAÑA,Jardines En La Montaña,TLALPAN,Tlalpan,CDMX,19.30086,-99.20877
1,2016.0,1.0,2016-01-01,0.0,2015.0,12.0,2015-12-31,22.0,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,...,INVESTIGACIÓN EN TLALPAN,TLP-1,UI-2CD,LOMAS DE PADIERNA,Lomas De Padierna,TLALPAN,Tlalpan,CDMX,19.29003,-99.21748
2,2016.0,1.0,2016-01-01,0.0,2016.0,1.0,2016-01-01,0.0,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,...,INVESTIGACIÓN EN IZTAPALAPA,IZP-2,UI-2CD,SAN ANTONIO CULHUACÁN,Barrio San Antonio Culhuacan,IZTAPALAPA,Iztapalapa,CDMX,19.34080,-99.11431
3,2016.0,1.0,2016-01-01,0.0,2015.0,12.0,2015-12-31,22.0,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,...,INVESTIGACIÓN EN GUSTAVO A. MADERO,GAM-8,UI-2CD,SAN JUAN DE ARAGÓN II SECCIÓN,San Juan De Aragon Ii Seccion,GUSTAVO A. MADERO,Gustavo A. Madero,CDMX,19.45106,-99.08669
4,2016.0,1.0,2016-01-01,0.0,2015.0,12.0,2015-12-31,22.0,HOMICIDIOS INTENCIONALES (OTROS),HOMICIDIO DOLOSO,...,INVESTIGACIÓN EN BENITO JUÁREZ,BJ-1,UI-2SD,NATIVITAS,Nativitas,BENITO JUAREZ,Benito Juárez,CDMX,19.38095,-99.13982


In [ ]:
le = LabelEncoder() # convertimos los valores categóricos a numéricos
df_c['colonia_catalogo'] = le.fit_transform(df_c['colonia_catalogo'])
df_c['alcaldia_catalogo'] = le.fit_transform(df_c['alcaldia_catalogo'])
df_c['municipio_hecho'] = le.fit_transform(df_c['municipio_hecho'])
df_c['fiscalia'] = le.fit_transform(df_c['fiscalia'])
df_c['agencia'] = le.fit_transform(df_c['agencia'])
df_c['competencia'] = le.fit_transform(df_c['competencia'])
df_c['unidad_investigacion'] = le.fit_transform(df_c['unidad_investigacion'])
df_c['delito'] = le.fit_transform(df_c['delito'])

In [ ]:
"""df_numerics['colonia_catalogo'] = df_c['colonia_catalogo']
df_numerics['alcaldia_catalogo'] = df_c['alcaldia_catalogo']
df_numerics['municipio_hecho'] = df_c['municipio_hecho']
df_numerics['fiscalia'] = df_c['fiscalia']
df_numerics['agencia'] = df_c['agencia']
df_numerics['competencia'] = df_c['competencia']
df_numerics['unidad_investigacion'] = df_c['unidad_investigacion']
df_numerics['delito'] = df_c['delito']"""


"df_numerics['colonia_catalogo'] = df_c['colonia_catalogo']\ndf_numerics['alcaldia_catalogo'] = df_c['alcaldia_catalogo']\ndf_numerics['municipio_hecho'] = df_c['municipio_hecho']\ndf_numerics['fiscalia'] = df_c['fiscalia']\ndf_numerics['agencia'] = df_c['agencia']\ndf_numerics['competencia'] = df_c['competencia']\ndf_numerics['unidad_investigacion'] = df_c['unidad_investigacion']\ndf_numerics['delito'] = df_c['delito']"

In [ ]:
df_c[['delito', 'competencia']].corr() #Calculamos la correlación de las variables categoricas conrespecto al delito

,delito,competencia
delito,1.00000,0.00495
competencia,0.00495,1.00000


In [ ]:
df_c[['delito', 'colonia_catalogo']].corr()

,delito,colonia_catalogo
delito,1.000000,-0.016703
colonia_catalogo,-0.016703,1.000000


In [ ]:
df_c[['delito', 'alcaldia_catalogo']].corr()

,delito,alcaldia_catalogo
delito,1.000000,-0.009757
alcaldia_catalogo,-0.009757,1.000000


In [ ]:
df_c[['delito', 'municipio_hecho']].corr()

,delito,municipio_hecho
delito,1.000000,-0.001079
municipio_hecho,-0.001079,1.000000


In [ ]:
df_c[['delito', 'fiscalia']].corr()

,delito,fiscalia
delito,1.000000,-0.032783
fiscalia,-0.032783,1.000000


In [ ]:
df_c[['delito', 'agencia']].corr()

,delito,agencia
delito,1.000000,-0.088724
agencia,-0.088724,1.000000


In [ ]:
df_c[['delito', 'unidad_investigacion']].corr()

,delito,unidad_investigacion
delito,1.000000,0.048299
unidad_investigacion,0.048299,1.000000


In [ ]:
# Al ver que son insignificantes, las quitamos
df.drop(columns=['colonia_catalogo', 'alcaldia_catalogo', 'municipio_hecho', 'fiscalia', 'agencia', 'competencia', 'unidad_investigacion']  ,axis=1,inplace=True)
df.head()

,anio_inicio,mes_inicio,fecha_inicio,hora_inicio,anio_hecho,mes_hecho,fecha_hecho,hora_hecho,delito,categoria_delito,colonia_hecho,alcaldia_hecho,latitud,longitud
0,2016.0,1.0,2016-01-01,0.0,2015.0,12.0,2015-12-31,16.0,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,DELITO DE BAJO IMPACTO,JARDINES EN LA MONTAÑA,TLALPAN,19.30086,-99.20877
1,2016.0,1.0,2016-01-01,0.0,2015.0,12.0,2015-12-31,22.0,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,LOMAS DE PADIERNA,TLALPAN,19.29003,-99.21748
2,2016.0,1.0,2016-01-01,0.0,2016.0,1.0,2016-01-01,0.0,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,SAN ANTONIO CULHUACÁN,IZTAPALAPA,19.34080,-99.11431
3,2016.0,1.0,2016-01-01,0.0,2015.0,12.0,2015-12-31,22.0,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,SAN JUAN DE ARAGÓN II SECCIÓN,GUSTAVO A. MADERO,19.45106,-99.08669
4,2016.0,1.0,2016-01-01,0.0,2015.0,12.0,2015-12-31,22.0,HOMICIDIOS INTENCIONALES (OTROS),HOMICIDIO DOLOSO,NATIVITAS,BENITO JUAREZ,19.38095,-99.13982


In [ ]:
#df_numerics.drop(columns=['competencia', 'alcaldia_catalogo', 'municipio_hecho', 'fiscalia', 'agencia', 'competencia', 'unidad_investigacion'], axis=1, inplace=True)

In [ ]:
df.isna().sum()   #Observamos los datos nulos que nos quedan

,0
anio_inicio,0
mes_inicio,0
fecha_inicio,0
hora_inicio,0
anio_hecho,0
mes_hecho,0
fecha_hecho,0
hora_hecho,0
delito,0
categoria_delito,0


In [ ]:
porcentaje_colonia = df['colonia_hecho'].isna().sum()/df.shape[0]       #calculamos los datos nulos entre el total de los datos para saber que porcentaje son
porcentaje_colonia

0.04819909319103459

In [ ]:
porcentaje_latitud = df['latitud'].isna().sum()/df.shape[0]
porcentaje_latitud

0.04774149280402412

In [ ]:
porcentaje_alcaldia = df['alcaldia_hecho'].isna().sum()/df.shape[0]
porcentaje_alcaldia

0.0026977196494198546

In [ ]:
#Eliminamos columnas que no nos dan informacion sobre prevenir delitos, pues nos hablan de cosas que pasan después.
df.drop( columns=['fecha_inicio', 'hora_inicio', 'mes_inicio', 'anio_inicio', 'anio_hecho', 'mes_hecho'] ,axis=1,inplace=True)

In [ ]:
df_clean = df.dropna() #Eliminamos los datos nulos
df_clean.index = range(len(df_clean))
df_clean.shape

(1927876, 8)

In [ ]:
df_clean.isna().sum()

,0
fecha_hecho,0
hora_hecho,0
delito,0
categoria_delito,0
colonia_hecho,0
alcaldia_hecho,0
latitud,0
longitud,0


In [ ]:
df_clean['hora_hecho'].value_counts()

,count
hora_hecho,
12.0,239597
10.0,132722
11.0,106099
14.0,101986
15.0,101613
13.0,95077
20.0,94628
16.0,94251
9.0,92899


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1927876 entries, 0 to 1927875
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   fecha_hecho       datetime64[ns]
 1   hora_hecho        float64       
 2   delito            object        
 3   categoria_delito  object        
 4   colonia_hecho     object        
 5   alcaldia_hecho    object        
 6   latitud           float64       
 7   longitud          float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 117.7+ MB


In [ ]:
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        df_clean[col] = df_clean[col].str.lower()

<ipython-input-47-9b28aa7aca5e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[col] = df_clean[col].str.lower()


In [ ]:
df_clean.head()

,fecha_hecho,hora_hecho,delito,categoria_delito,colonia_hecho,alcaldia_hecho,latitud,longitud
0,2015-12-31,16.0,lesiones culposas por transito vehicular en co...,delito de bajo impacto,jardines en la montaña,tlalpan,19.30086,-99.20877
1,2015-12-31,22.0,robo a pasajero a bordo de taxi con violencia,robo a pasajero a bordo de taxi con violencia,lomas de padierna,tlalpan,19.29003,-99.21748
2,2016-01-01,0.0,robo a transeunte en via publica con violencia,robo a transeunte en vía pública con y sin vio...,san antonio culhuacán,iztapalapa,19.34080,-99.11431
3,2015-12-31,22.0,robo de vehiculo de servicio particular sin vi...,robo de vehículo con y sin violencia,san juan de aragón ii sección,gustavo a. madero,19.45106,-99.08669
4,2015-12-31,22.0,homicidios intencionales (otros),homicidio doloso,nativitas,benito juarez,19.38095,-99.13982


In [ ]:
columns_to_tokenize = df_clean.select_dtypes(include=['object']).columns    #Queremos tokenizar columnas con datos categoricos

for column in columns_to_tokenize:
    df_clean.loc[:, column] = df_clean[column].apply(lambda x: word_tokenize(x) if isinstance(x, str) else x)

df_clean.head()

,fecha_hecho,hora_hecho,delito,categoria_delito,colonia_hecho,alcaldia_hecho,latitud,longitud
0,2015-12-31,16.0,"[lesiones, culposas, por, transito, vehicular,...","[delito, de, bajo, impacto]","[jardines, en, la, montaña]",[tlalpan],19.30086,-99.20877
1,2015-12-31,22.0,"[robo, a, pasajero, a, bordo, de, taxi, con, v...","[robo, a, pasajero, a, bordo, de, taxi, con, v...","[lomas, de, padierna]",[tlalpan],19.29003,-99.21748
2,2016-01-01,0.0,"[robo, a, transeunte, en, via, publica, con, v...","[robo, a, transeunte, en, vía, pública, con, y...","[san, antonio, culhuacán]",[iztapalapa],19.34080,-99.11431
3,2015-12-31,22.0,"[robo, de, vehiculo, de, servicio, particular,...","[robo, de, vehículo, con, y, sin, violencia]","[san, juan, de, aragón, ii, sección]","[gustavo, a., madero]",19.45106,-99.08669
4,2015-12-31,22.0,"[homicidios, intencionales, (, otros, )]","[homicidio, doloso]",[nativitas],"[benito, juarez]",19.38095,-99.13982


In [ ]:
df_clean.head()

,fecha_hecho,hora_hecho,delito,categoria_delito,colonia_hecho,alcaldia_hecho,latitud,longitud
0,2015-12-31,16.0,"[lesiones, culposas, por, transito, vehicular,...","[delito, de, bajo, impacto]","[jardines, en, la, montaña]",[tlalpan],19.30086,-99.20877
1,2015-12-31,22.0,"[robo, a, pasajero, a, bordo, de, taxi, con, v...","[robo, a, pasajero, a, bordo, de, taxi, con, v...","[lomas, de, padierna]",[tlalpan],19.29003,-99.21748
2,2016-01-01,0.0,"[robo, a, transeunte, en, via, publica, con, v...","[robo, a, transeunte, en, vía, pública, con, y...","[san, antonio, culhuacán]",[iztapalapa],19.34080,-99.11431
3,2015-12-31,22.0,"[robo, de, vehiculo, de, servicio, particular,...","[robo, de, vehículo, con, y, sin, violencia]","[san, juan, de, aragón, ii, sección]","[gustavo, a., madero]",19.45106,-99.08669
4,2015-12-31,22.0,"[homicidios, intencionales, (, otros, )]","[homicidio, doloso]",[nativitas],"[benito, juarez]",19.38095,-99.13982


In [ ]:
def primera_palabra(tokens):    # Tokenizamos estos valores
    return tokens[0] if isinstance(tokens, list) and len(tokens) > 0 else None

df_clean['tipo_delito'] = df_clean['delito'].apply(primera_palabra) #Creamos nueva columna con tipo de delitos, generalizando palabras en comun de delito

df_clean[['delito', 'tipo_delito']].head()

<ipython-input-52-cdc1bc9f9d77>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['tipo_delito'] = df_clean['delito'].apply(primera_palabra)


,delito,tipo_delito
0,"[lesiones, culposas, por, transito, vehicular,...",lesiones
1,"[robo, a, pasajero, a, bordo, de, taxi, con, v...",robo
2,"[robo, a, transeunte, en, via, publica, con, v...",robo
3,"[robo, de, vehiculo, de, servicio, particular,...",robo
4,"[homicidios, intencionales, (, otros, )]",homicidios


In [ ]:
df_clean['tipo_delito'].value_counts()

,count
tipo_delito,
robo,774516
violencia,244193
fraude,143146
amenazas,128599
daño,93132
...,...
privación,2
injurias,1
utilizacion,1


In [ ]:
df_clean['tipo_delito'] = df_clean['tipo_delito'].replace('la', 'corrupcion')     #Convertimos las palabras iniciales que no brindan informacion sobre el delito
df_clean['tipo_delito'] = df_clean['tipo_delito'].replace('perdida', 'accidente')
df_clean['tipo_delito'] = df_clean['tipo_delito'].replace('contra', 'corrupcion')

<ipython-input-55-a4e2321ff7c2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['tipo_delito'] = df_clean['tipo_delito'].replace('la', 'corrupcion')
<ipython-input-55-a4e2321ff7c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['tipo_delito'] = df_clean['tipo_delito'].replace('perdida', 'accidente')
<ipython-input-55-a4e2321ff7c2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [ ]:
x= list(df_clean['tipo_delito'].values); x.count('corrupcion')

x = list(df_clean['tipo_delito'].value_counts().index)
y = list(df_clean['tipo_delito'].value_counts())

print(x[:10])
print(y[:10])


['robo', 'violencia', 'fraude', 'amenazas', 'daño', 'abuso', 'lesiones', 'narcomenudeo', 'denuncia', 'usurpación']
[774516, 244193, 143146, 128599, 93132, 72417, 72017, 38647, 36534, 34196]


In [ ]:
unicas_x = list(df_clean['tipo_delito'].unique())
print(unicas_x)


['lesiones', 'robo', 'homicidios', 'accidente', 'homicidio', 'daño', 'fraude', 'denuncia', 'corrupcion', 'negacion', 'resistencia', 'amenazas', 'encubrimiento', 'abuso', 'falsificacion', 'usurpación', 'allanamiento', 'falsedad', 'despojo', 'sustraccion', 'ataque', 'delitos', 'tentativa', 'aborto', 'narcomenudeo', 'insolvencia', 'uso', 'ejercicio', 'cohecho', 'quebrantamiento', 'portacion', 'responsabilidad', 'bigamia', 'pornografia', 'desobedencia', 'disparos', 'abandono', 'usurpacion', 'posesion', 'feminicidio', 'maltrato', 'oposicion', 'revelacion', 'inhumaciones', 'acoso', 'operaciones', 'peligro', 'secuestro', 'trafico', 'explotación', 'evasion', 'violacion', 'variacion', 'peculado', 'motin', 'enriquecimiento', 'intimidacion', 'coaccion', 'provocacion', 'difamacion', 'gestión', 'concusion', 'asociacion', 'contaminación', 'tala', 'regulación', 'cambio', 'desacato', 'ultrajes', 'ataques', 'procreacion', 'otros', 'sabotaje', 'calumnias', 'lenocinio', 'cobranza', 'personas', 'producció

In [ ]:
len(unicas_x)   #Vemos el número de tipos de delitos que obtenemos

110

In [ ]:
df_clean['tipo_delito'].value_counts()

,count
tipo_delito,
robo,774516
violencia,244193
fraude,143146
amenazas,128599
daño,93132
...,...
calumnias,2
utilizacion,1
contagio,1


In [ ]:
print(x[:15])
print(y[:15])

['robo', 'violencia', 'fraude', 'amenazas', 'daño', 'abuso', 'lesiones', 'narcomenudeo', 'denuncia', 'usurpación', 'despojo', 'falsificacion', 'tentativa', 'accidente', 'corrupcion']
[774516, 244193, 143146, 128599, 93132, 72417, 72017, 38647, 36534, 34196, 33308, 28152, 21338, 20952, 17881]


In [ ]:
df_clean.head(35)

,fecha_hecho,hora_hecho,delito,categoria_delito,colonia_hecho,alcaldia_hecho,latitud,longitud,tipo_delito
0,2015-12-31,16.0,"[lesiones, culposas, por, transito, vehicular,...","[delito, de, bajo, impacto]","[jardines, en, la, montaña]",[tlalpan],19.30086,-99.20877,lesiones
1,2015-12-31,22.0,"[robo, a, pasajero, a, bordo, de, taxi, con, v...","[robo, a, pasajero, a, bordo, de, taxi, con, v...","[lomas, de, padierna]",[tlalpan],19.29003,-99.21748,robo
2,2016-01-01,0.0,"[robo, a, transeunte, en, via, publica, con, v...","[robo, a, transeunte, en, vía, pública, con, y...","[san, antonio, culhuacán]",[iztapalapa],19.34080,-99.11431,robo
3,2015-12-31,22.0,"[robo, de, vehiculo, de, servicio, particular,...","[robo, de, vehículo, con, y, sin, violencia]","[san, juan, de, aragón, ii, sección]","[gustavo, a., madero]",19.45106,-99.08669,robo
4,2015-12-31,22.0,"[homicidios, intencionales, (, otros, )]","[homicidio, doloso]",[nativitas],"[benito, juarez]",19.38095,-99.13982,homicidios
5,2015-12-31,14.0,"[robo, a, transeunte, en, via, publica, con, v...","[robo, a, transeunte, en, vía, pública, con, y...","[culhuacán, ctm, sección, vii]",[coyoacan],19.32651,-99.11497,robo
6,2015-12-31,17.0,"[robo, de, objetos]","[delito, de, bajo, impacto]","[leyes, de, reforma, 3a, sección]",[iztapalapa],19.38117,-99.07182,robo
7,2015-12-31,23.0,"[lesiones, intencionales, por, arma, de, fuego]","[lesiones, dolosas, por, disparo, de, arma, de...",[providencia],"[gustavo, a., madero]",19.48364,-99.07066,lesiones
8,2015-12-31,21.0,"[perdida, de, la, vida, por, otras, causas]","[hecho, no, delictivo]",[buenavista],[iztapalapa],19.32098,-99.02693,accidente
9,2016-01-01,1.0,"[lesiones, culposas, por, transito, vehicular,...","[delito, de, bajo, impacto]","[cerro, de, la, estrella]",[iztapalapa],19.31861,-99.07572,lesiones


In [ ]:
df_dates = df_clean.copy()

In [ ]:
def es_fin_de_semana(fecha):        # Función para verificar si es fin de semana
    return fecha.weekday() >= 5

def es_cercano_a_quincena(fecha):   # Función para verificar si es día cercano a la quincena
    return fecha.day in [14, 15, 16, 29, 30, 31, 1]

                                    # Agregar columnas booleanas al DataFrame sobre las fechas
df_dates['es_fin_de_semana'] = df_dates['fecha_hecho'].apply(es_fin_de_semana).astype(int)
df_dates['es_cercano_a_quincena'] = df_dates['fecha_hecho'].apply(es_cercano_a_quincena).astype(int)

df_dates.head()

,fecha_hecho,hora_hecho,delito,categoria_delito,colonia_hecho,alcaldia_hecho,latitud,longitud,tipo_delito,es_fin_de_semana,es_cercano_a_quincena
0,2015-12-31,16.0,"[lesiones, culposas, por, transito, vehicular,...","[delito, de, bajo, impacto]","[jardines, en, la, montaña]",[tlalpan],19.30086,-99.20877,lesiones,0,1
1,2015-12-31,22.0,"[robo, a, pasajero, a, bordo, de, taxi, con, v...","[robo, a, pasajero, a, bordo, de, taxi, con, v...","[lomas, de, padierna]",[tlalpan],19.29003,-99.21748,robo,0,1
2,2016-01-01,0.0,"[robo, a, transeunte, en, via, publica, con, v...","[robo, a, transeunte, en, vía, pública, con, y...","[san, antonio, culhuacán]",[iztapalapa],19.34080,-99.11431,robo,0,1
3,2015-12-31,22.0,"[robo, de, vehiculo, de, servicio, particular,...","[robo, de, vehículo, con, y, sin, violencia]","[san, juan, de, aragón, ii, sección]","[gustavo, a., madero]",19.45106,-99.08669,robo,0,1
4,2015-12-31,22.0,"[homicidios, intencionales, (, otros, )]","[homicidio, doloso]",[nativitas],"[benito, juarez]",19.38095,-99.13982,homicidios,0,1


In [ ]:
df_dates['es_fin_de_semana'].value_counts()

,count
es_fin_de_semana,
0,1446320
1,481556


In [ ]:
df_dates['es_cercano_a_quincena'].value_counts()

,count
es_cercano_a_quincena,
0,1504473
1,423403


In [ ]:
df_dates['fecha_hecho'].value_counts()

,count
fecha_hecho,
2021-01-01,1195
2020-01-01,1077
2016-09-04,987
2018-06-01,935
2019-04-01,896
...,...
2010-05-18,1
1983-08-05,1
1980-03-06,1


In [ ]:
df_dates = df_dates[(df_clean['fecha_hecho'] >= '2016-01-01') & (df_dates['fecha_hecho'] <= '2024-12-31')] #Nos aseguramos que las fechas sean del rango correcto

df_dates.head()

,fecha_hecho,hora_hecho,delito,categoria_delito,colonia_hecho,alcaldia_hecho,latitud,longitud,tipo_delito,es_fin_de_semana,es_cercano_a_quincena
2,2016-01-01,0.0,"[robo, a, transeunte, en, via, publica, con, v...","[robo, a, transeunte, en, vía, pública, con, y...","[san, antonio, culhuacán]",[iztapalapa],19.34080,-99.11431,robo,0,1
9,2016-01-01,1.0,"[lesiones, culposas, por, transito, vehicular,...","[delito, de, bajo, impacto]","[cerro, de, la, estrella]",[iztapalapa],19.31861,-99.07572,lesiones,0,1
11,2016-01-01,1.0,"[daño, en, propiedad, ajena, intencional, a, a...","[delito, de, bajo, impacto]",[popotla],"[miguel, hidalgo]",19.45716,-99.17253,daño,0,1
14,2016-01-01,1.0,"[robo, a, casa, habitacion, sin, violencia]","[delito, de, bajo, impacto]","[los, cedros]",[coyoacan],19.31267,-99.11181,robo,0,1
15,2016-01-01,2.0,"[daño, en, propiedad, ajena, intencional, a, n...","[delito, de, bajo, impacto]","[santa, maria, malinalco]",[azcapotzalco],19.47936,-99.17994,daño,0,1


In [ ]:
df_dates['fecha_hecho'].value_counts()

,count
fecha_hecho,
2021-01-01,1195
2020-01-01,1077
2016-09-04,987
2018-06-01,935
2019-04-01,896
...,...
2022-12-25,288
2020-04-26,285
2024-09-29,261


In [ ]:
df_dates.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1898450 entries, 2 to 1927875
Data columns (total 11 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   fecha_hecho            datetime64[ns]
 1   hora_hecho             float64       
 2   delito                 object        
 3   categoria_delito       object        
 4   colonia_hecho          object        
 5   alcaldia_hecho         object        
 6   latitud                float64       
 7   longitud               float64       
 8   tipo_delito            object        
 9   es_fin_de_semana       int64         
 10  es_cercano_a_quincena  int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 173.8+ MB


In [ ]:
df_dates['year'] = df_dates['fecha_hecho'].astype(str).str[:4]

df_dates['year'].value_counts()

<ipython-input-98-e6611827b592>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dates['year'] = df_dates['fecha_hecho'].astype(str).str[:4]


,count
year,
2018,245565
2019,236877
2022,223912
2023,222822
2017,218614
2021,216193
2020,197166
2016,187941
2024,149360


In [ ]:
maximos_minimos = df_dates.describe().loc[['min', 'max']]

print(maximos_minimos)

             fecha_hecho  hora_hecho   latitud    longitud  es_fin_de_semana  \
min  2016-01-01 00:00:00         0.0  19.09535 -100.232489               0.0   
max  2024-09-30 00:00:00        23.0  19.58333  -98.946860               1.0   

     es_cercano_a_quincena  
min                    0.0  
max                    1.0  


In [ ]:
df_dates.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1898450 entries, 2 to 1927875
Data columns (total 13 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   fecha_hecho            datetime64[ns]
 1   hora_hecho             float64       
 2   delito                 object        
 3   categoria_delito       object        
 4   colonia_hecho          object        
 5   alcaldia_hecho         object        
 6   latitud                float64       
 7   longitud               float64       
 8   tipo_delito            object        
 9   es_fin_de_semana       int64         
 10  es_cercano_a_quincena  int64         
 11  anio                   object        
 12  year                   object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(7)
memory usage: 202.8+ MB


In [ ]:
df_dates.drop(columns=['anio'], axis=1, inplace=True)

<ipython-input-102-88681487b449>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dates.drop(columns=['anio'], axis=1, inplace=True)


In [ ]:
df_dates['year'] = df_dates['year'].astype(int)

<ipython-input-103-b82596e81bc3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dates['year'] = df_dates['year'].astype(int)


In [ ]:
df_dates.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1898450 entries, 2 to 1927875
Data columns (total 12 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   fecha_hecho            datetime64[ns]
 1   hora_hecho             float64       
 2   delito                 object        
 3   categoria_delito       object        
 4   colonia_hecho          object        
 5   alcaldia_hecho         object        
 6   latitud                float64       
 7   longitud               float64       
 8   tipo_delito            object        
 9   es_fin_de_semana       int64         
 10  es_cercano_a_quincena  int64         
 11  year                   int64         
dtypes: datetime64[ns](1), float64(3), int64(3), object(5)
memory usage: 188.3+ MB


In [ ]:
df_dates['month'] = df_dates['fecha_hecho'].astype(str).str[5:7].astype(int)    Separamos fecha en año, mes y dia
df_dates['day'] = df_dates['fecha_hecho'].astype(str).str[-2:].astype(int)
df_dates['month'].value_counts()

<ipython-input-106-805574b63992>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dates['month'] = df_dates['fecha_hecho'].astype(str).str[5:7].astype(int)
<ipython-input-106-805574b63992>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dates['day'] = df_dates['fecha_hecho'].astype(str).str[-2:].astype(int)


,count
month,
3,172302
5,167857
8,165625
6,160901
7,160476
1,159300
2,158762
9,158746
4,157694


In [ ]:
df_dates['day'].value_counts()

,count
day,
1,73627
15,69616
10,64713
20,64149
5,63667
12,62649
4,62563
3,62546
17,62512


In [ ]:
df_dates.to_csv('df_dates.csv', index=False)   #Exportamos el dataframe limpio

In [ ]:
df_dates['alcaldia_hecho'].value_counts()

,count
alcaldia_hecho,
[cuauhtemoc],280884
[iztapalapa],280191
"[gustavo, a., madero]",194092
"[benito, juarez]",153665
[coyoacan],133161
"[alvaro, obregon]",130181
"[miguel, hidalgo]",122167
[tlalpan],117095
"[venustiano, carranza]",110420
